In [1]:
import openresim as rs
import numpy as np

In [2]:
def create_model_example_7_1():
    grid = rs.grids.Cartesian(
        nx=4, ny=1, nz=1, dx=300, dy=350, dz=40, phi=0.27, kx=270, dtype="double"
    )
    fluid = rs.fluids.SinglePhase(mu=0.5, B=1, dtype="double")
    model = rs.models.Model(grid, fluid, dtype="double", verbose=False)
    model.set_well(id=4, q=-600, s=1.5, r=3.5)
    model.set_boundaries({0: ("pressure", 4000), 5: ("rate", 0)})
    return model


def create_model():
    grid = rs.grids.Cartesian(
        nx=3,
        ny=1,
        nz=1,
        dx=300,
        dy=350,
        dz=20,
        phi=0.27,
        kx=1,
        ky=1,
        kz=0.1,
        comp=1 * 10**-6,
        dtype="double",
    )
    fluid = rs.fluids.SinglePhase(
        mu=0.5, B=1, rho=50, comp=1 * 10**-5, dtype="double"
    )
    model = rs.models.Model(
        grid, fluid, pi=6000, dt=5, start_date="10.10.2018", dtype="double"
    )

    n_samples = 1
    cells_id = model.grid.get_cells_id(False, True)[-1].flatten()
    wells = np.random.choice(cells_id, n_samples, False)
    for id in wells:
        model.set_well(id=id, q=-300, pwf=100, s=1.5, r=3.5)

    wells = np.random.choice(cells_id, n_samples, False)
    for id in wells:
        model.set_well(id=id, q=100, s=0, r=3.5)

    return model


model = create_model()
model.run(10, isolver="cgs")

[info] Simulation run started: 10 timesteps.


[step]: 10 - : 100%|██████████| 10/10 [00:00<00:00, 375.70steps/s]

[info] Simulation run of 10 steps finished in 0.03 seconds.
[info] Material Balance Error: 2.55351295663786e-15.


In [3]:
arr = model.grid.get_cells_center(True, False)
print(arr.shape)
print(arr.min(axis=0))
arr

(5, 3)
[150. 175.  10.]


array([[ 150.,  175.,   10.],
       [ 450.,  175.,   10.],
       [ 750.,  175.,   10.],
       [1050.,  175.,   10.],
       [1350.,  175.,   10.]])

In [4]:
scaler = rs.scalers.MinMaxScaler((-1, 1)).fit(arr, axis=0)
scaler.transform(arr)

c:\users\abugrzka\projects\openresim\openresim\scalers.py:60: RuntimeWarning: invalid value encountered in true_divide
  self.vmax - self.vmin


array([[-1. ,  nan,  nan],
       [-0.5,  nan,  nan],
       [ 0. ,  nan,  nan],
       [ 0.5,  nan,  nan],
       [ 1. ,  nan,  nan]])

In [5]:
t = np.arange(0, (model.tstep + 1) * model.dt, model.dt)
rs.scalers.MinMaxScaler((0, 1)).fit_transform(t.reshape(-1, 1), axis=0)

array([[0. ],
       [0.1],
       [0.2],
       [0.3],
       [0.4],
       [0.5],
       [0.6],
       [0.7],
       [0.8],
       [0.9],
       [1. ]])

In [6]:
t

array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50])

In [7]:
t, x = model.get_domain(False, True)
t, x

(array([[ 0],
        [ 5],
        [10],
        [15],
        [20],
        [25],
        [30],
        [35],
        [40],
        [45],
        [50]]),
 array([[ 150.],
        [ 450.],
        [ 750.],
        [1050.],
        [1350.]]))

In [8]:
rs.scalers.MinMaxScaler((-1, 1)).fit_transform(x, axis=0)

array([[-1. ],
       [-0.5],
       [ 0. ],
       [ 0.5],
       [ 1. ]])

In [9]:
t

array([[ 0],
       [ 5],
       [10],
       [15],
       [20],
       [25],
       [30],
       [35],
       [40],
       [45],
       [50]])

In [10]:
rs.scalers.MinMaxScaler((0, 1)).fit_transform(t.flatten(), axis=0)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [11]:
model.get_centers(False)

(array([[ 150.],
        [ 450.],
        [ 750.],
        [1050.],
        [1350.]]),
 ['x'])

In [12]:
t

array([[ 0],
       [ 5],
       [10],
       [15],
       [20],
       [25],
       [30],
       [35],
       [40],
       [45],
       [50]])

In [13]:
np.repeat(t.flatten(), 2)

array([ 0,  0,  5,  5, 10, 10, 15, 15, 20, 20, 25, 25, 30, 30, 35, 35, 40,
       40, 45, 45, 50, 50])

In [14]:
config = {"time": ["MinMax", (0, 1)], "space": ["minMax"]}

In [18]:
model.set_scalers({"time": ["minmax", (-10, 10)], "rate": ["minmax", None]})
model.scalers_dict

{'time': ['MinMax', (-10, 10)],
 'space': ['MinMax', (-1, 1)],
 'pressure': ['MinMax', (-1, 1)],
 'rate': [None, None]}

In [16]:
model.update_scalers(False)
model.pressures_scaler.transform(model.pressures[1])
# model.space_scaler.transform(model.get_centers(False)[0])

array([-1.        , -0.69223375, -0.94898548, -0.9902352 , -1.        ])

In [17]:
model.get_df(
    scale=True,
    melt=True,
    boundary=False,
    units=True,
    columns=["time", "cells", "wells"],
)

,id,Step,x,Time [Scaled],Q [Scaled],P [Scaled]
0,1,0,-0.5,-10.0,0.0,-1.000000
1,2,0,0.0,-10.0,0.0,-1.000000
2,3,0,0.5,-10.0,0.0,-1.000000
3,1,1,-0.5,-8.0,100.0,-0.692234
4,2,1,0.0,-8.0,0.0,-0.948985
5,3,1,0.5,-8.0,0.0,-0.990235
6,1,2,-0.5,-6.0,100.0,-0.427903
7,2,2,0.0,-6.0,0.0,-0.868141
8,3,2,0.5,-6.0,0.0,-0.966865
9,1,3,-0.5,-4.0,100.0,-0.195355
